In [1]:
import numpy as np
from maxvolpy.maxvol import maxvol
from numpy.polynomial import Chebyshev as T
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.linalg import lu as lu
from gen_1D import *
from ids_pullback import *
#from ids import *
from plu import *
from blocked_maxvol import *
import scipy.linalg
%matplotlib inline

In [2]:
### how to make from permatation matrix readable array
def perm_array(A):
    p_a = np.array((A.shape[0]))
    p_a = np.argmax(A, axis = 1)
    return p_a

def perm_matrix(p, m = 'P'):
    p_m = np.zeros((p.shape[0],p.shape[0]),dtype=float)
    if m == 'P':
        for i in range(0,p.shape[0]):     
            p_m[i,p[i]] = 1.0
    if m == 'Q':
        for i in range(0,p.shape[0]):     
            p_m[p[i],i] = 1.0
        
    return p_m

def form_permute(C, j, ind):
    C[ind],C[j]=C[j],C[ind]
    return()  

def p_preproc(p,ndim):
    loc = np.copy(p)
    for j in range(0,len(loc),ndim):
        loc[j:j+ndim] = np.sort(loc[j:j+ndim])
    return(loc)

def change_intersept(inew, iold, full=True):
    """
    change two sets of rows or columns when indices may intersept with preserving order
    RETURN two sets of indices,
    than say A[idx_n] = A[idx_o]
    """
    union = np.array(list( set(inew) | set(iold) ))
    idx_n = np.hstack((inew, np.setdiff1d(union, inew)))
    idx_o = np.hstack((iold, np.setdiff1d(union, iold)))
    return  idx_n, idx_o

def pluq_ids(A, nder = 1, debug = False):

    def mov_LU(C, j, ind_r, ind_c, m = 'U'):
        if m == 'U':
            C[[ind_r,j],:] = C[[j,ind_r],:]
            C[:,[ind_c,j]] = C[:,[j,ind_c]]
        if m=='L':
            temp = np.copy(C[ind_r,:j])
            C[ind_r,:j] = C[j, :j]
            C[j, :j] = temp

            temp = np.copy(C[:j,ind_c])
            C[:j, ind_c] = C[:j,j]
            C[:j, j] = temp  
            
    def elimination(L,U,ind):
        k = L.shape[0]
        for i in range(ind+1, k):
            L[i,ind] = U[i,ind]/U[ind,ind]
            U[i,ind:] -= L[i,ind]*U[ind,ind:] 
        return ()    
    
    def restore_lu(L,U,ind):
        k = L.shape[0]
        for i in range(ind+1, k):
            U[i,ind:] += L[i,ind]*U[ind,ind:]
        return (U)
    
    def restore_layer(L,U,ind, ndim):
        k = L.shape[0]
        for j in range(ndim-1,ind-1, -1):
            for i in range(j+1, k):
                U[i,ind:] += L[i,ind]*U[ind,ind:]    
        return (U)
    
    def det_search(A,start_ind1, start_ind2):
        det = 0.0
        row = start_ind1 

        for k in range(start_ind1,A.shape[0],ndim):
            if k not in black_list:
                pair = A[k:k+ndim][:,start_ind2:].T
                #print pair, np.linalg.matrix_rank(pair)
                _,y,_ = scipy.linalg.svd(pair)
                ra = 0
                #print y
                for t in range(0,len(y)):
                    if np.abs(y[t]) > 1e-20:
                        #print ra
                        ra = ra + 1
                if ra == ndim :
                    piv,_ = maxvol(pair)
                    if np.abs(np.linalg.det(pair[piv])) > det:
                        det, row = np.abs(np.linalg.det(pair[piv])), k
                
        return(det, row)        
                
    n, m = A.shape[0], A.shape[1]
    P = np.arange(n)
    L = np.eye(n, m, dtype=float)
    U = np.copy(A)
    Q = np.arange(m)
    ndim = nder + 1
    yx = np.array([0, 0], dtype=int)
    black_list = []
    info = np.zeros((2), dtype=int)
    treshold = 1e-10
    j = 0
    while (j < m):
        ### in each slice we are looking for 2x2 matrix with maxvol and memorize 'k' as a number of first row 
        ### 'k'- pair counter, which starts from current j position till the bottom of the matrix
        
        max_det, row_n = det_search(U,j,j)
        if (max_det == 0.0) and (j == 0):
            ### Critical error = no appropriate pair
            info[0] = 1
            return (P,L,U,Q,info)            
        if max_det == 0.0:
            if debug == True:
                print('error found')
            info[1] += 1
            j = j - ndim
            restore_layer(L,U,j,ndim)
            if debug:
            	print ('restored matrix')
            	print (U)
            max_det, row_n = det_search(U,j+ndim,j)
            if max_det == 0.0:
                # Critical error = all elements are in blacklist
                info[0] = 1
                return (P,L,U,Q,info)                
            black_list.append(row_n)
        #pair = U[row_n:row_n + ndim][:,j:].T
        loc_point = np.rot90(U[row_n:row_n + ndim][:,j:],1,(1,0))
        piv,_ = maxvol(loc_point)
        piv.sort()
        #print np.rot90(loc_point[piv],1,(0,1))
        #print ('check indx')
        #print piv + j,np.arange(ndim) + j
        indx_n, indx_o = change_intersept(np.arange(ndim) + j,piv + j)
        U[:,indx_n] = U[:,indx_o]
        Q[indx_n] = Q[indx_o]
        #print U[:ndim,:]
        #print Q
        indx_n, indx_o = change_intersept(np.arange(ndim) + j,row_n + np.arange(ndim))
        #print ('before rows swap')
        #print U
        U[indx_n,:] = U[indx_o,:]
        P[indx_n] = P[indx_o]
        #print ('after')
        #print U[:ndim]
        block = U[j:j+ndim,j:j+ndim]

        p_loc,l,rt = lu(block)
        print la.det(rt)
        p_loc  = perm_array(p_loc)
        #print ('p_locs')
        #print p_loc,np.arange(ndim)+j,(np.arange(ndim)+j)[p_loc]
        indx_n, indx_o = change_intersept(np.arange(ndim)+j,(np.arange(ndim)+j)[p_loc])
        #print indx_n,indx_o
        aab = np.concatenate((indx_n,indx_o))
        bba = np.concatenate((indx_o,indx_n))
        #print aab,bba
        
        U[aab,:] = U[bba,:]
    
        #print U[:ndim]
        P[indx_n] = P[indx_o]
        
        print P[:ndim]
        
        if (debug):
            if np.linalg.det(block) == max_det:
                print('correct 2x2 matrix')
              
            print ('on the', j, 'slice')
            print ('best row block is', row_n, row_n + 1)
            print ('column coordinates:', piv + j)
            print ('maxvol 2x2 submatrix', block)
            print ('with det = ', max_det)
            print ('pivoting and permutations start...')
       


        if debug == True:
            print ('just before elim')
            #print U
        if (debug):
            print('Elimination starts')
        ### make them all zeros! Below (j,j) element
        for idx in range(ndim):                      
                    
            elimination(L,U,j + idx)
            if (debug):
                print('after {} elimination'.format(idx))
                #print U

        j = j + ndim
        
    return(P,L,U,Q,info)  

In [6]:
n = 900 #nmber of sampling points   
deg = 11 #Degree of polynomiapaired plu Last Checkpoint: 22 minutes ago (unsaved changes)l          
interval = [-1, 1]  # interval of x
x = np.linspace(interval[0], interval[1], n, dtype=np.float64)
p_size = deg+1
nder = 1

In [4]:
### generating matrix with special structure
b = Gen_1D_coupled(p_size, x, debug=True)

number of points(l) = 900,  number of monoms(n_size) = 12


In [2]:
b = np.load('awsas.npy')
print b

[[  1.00000000e+00   9.73807854e-01   9.88330638e-01   9.97078392e-01]
 [  0.00000000e+00   8.92998383e+00   3.98831357e+00   1.00000000e+00]
 [  1.00000000e+00  -1.86628858e-02  -9.99922591e-01   6.22128299e-03]
 ..., 
 [  0.00000000e+00   5.32408894e+00   3.33148394e+00   1.00000000e+00]
 [  1.00000000e+00  -5.90449803e-01  -9.12649102e-01   2.08986720e-01]
 [  0.00000000e+00  -2.47589461e+00   8.35946878e-01   1.00000000e+00]]


In [19]:
print la.det(b[:b.shape[1]])
print la.matrix_rank(b)
_,s,_ = scipy.linalg.svd(b)
print s

1.01319149467e-08
10
[  8.03086424e+02   4.48638093e+02   3.26720909e+02   2.34634886e+02
   1.26669199e+02   5.55768893e+01   4.17381062e+01   2.53003032e+00
   6.50679779e-02   5.75393989e-04]


In [5]:
piv,_ = maxvol(b)
print piv
print la.det(b[piv])

[   0  243 3425 3731 1417 3605 2353 3573 2693  753]
-2.67815852211


In [8]:
p,l,u,q,inf = pluq_ids(b,nder, debug=True)

76.9617837065
[1416 1417]
correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 1416, 1417)
('column coordinates:', array([0, 9], dtype=int32))
('maxvol 2x2 submatrix', array([[  1.        ,   0.85018595],
       [  0.        ,  76.96178371]]))
('with det = ', 76.961783706549255)
pivoting and permutations start...
just before elim
Elimination starts
after 0 elimination
after 1 elimination
24.4819239383
[1416 1417]
correct 2x2 matrix
('on the', 2, 'slice')
('best row block is', 500, 501)
('column coordinates:', array([4, 7], dtype=int32))
('maxvol 2x2 submatrix', array([[ -1.96402959,  -0.33157009],
       [ -0.85846316, -12.61007717]]))
('with det = ', 24.481923938266984)
pivoting and permutations start...
just before elim
Elimination starts
after 0 elimination
after 1 elimination
-23.3432898347
[1416 1417]
('on the', 4, 'slice')
('best row block is', 1498, 1499)
('column coordinates:', array([4, 6], dtype=int32))
('maxvol 2x2 submatrix', array([[-2.2922111 , -0.53883573],
  

In [9]:
print la.det(u[:u.shape[1]])
de = np.dot(u, perm_matrix(q))
pde = np.dot(perm_matrix(p),de)
print la.det(np.dot(perm_matrix(p),b)[:b.shape[1]])

0.000164075408411
-0.000164075408456


In [10]:
p1,l1,u1,q1,inf = pluq_ids_pull(b, debug = True)

('Jumped on the layer', 0)
black_list now contains entries = 1
correct 2x2 matrix
('on the', 0, 'slice; curr_layer = ', 0)
('best row block is', 1416, 1417)
('column coordinates:', 0, 9)
('maxvol 2x2 submatrix', array([[  1.        ,   0.        ],
       [  0.85018595,  76.96178371]]))
('with det = ', 76.961783706549255)
pivoting and permutations start...
after 1st pivot
[[  1.           0.99810292   0.99241889 ...,   0.90844601   0.88098728
    0.85018595]
 [  0.           1.           3.56239276 ...,  -2.50337089 -10.4414753
  -17.70448803]
 [  1.           0.5007567   -0.49848545 ...,   0.50528885  -0.49393269
   -0.99996906]
 ..., 
 [  0.           1.           1.35546421 ...,   5.5856661   -3.12158032
   -9.5616153 ]
 [  1.           0.73599952   0.08339058 ...,   0.47343971   0.94475495
    0.91723867]
 [  0.           1.           2.94399807 ...,  -9.10775031  -3.87339638
    5.29562448]]
after 2nd pivot
[[  1.           0.85018595   0.99241889 ...,   0.90844601   0.88098728
  

In [11]:
print(la.det(np.dot(perm_matrix(p),b)[:b.shape[1]]))
print(la.det(np.dot(perm_matrix(p1),b)[:b.shape[1]]))
print p[:b.shape[1]],p1[:b.shape[1]]
print q,q1

-0.000164075408456
0.000164075408456
[1416 1417  500  501 1498 1499  386  387 3425 3424] [1416 1417  500  501 1498 1499  386  387 3424 3425]
[0 9 4 7 2 6 5 8 3 1] [0 9 4 7 2 6 5 8 3 1]


In [12]:
p = p_preproc(p,2)
print p[:b.shape[1]]
print la.det(np.dot(perm_matrix(p),b)[:b.shape[1]])

[1416 1417  500  501 1498 1499  386  387 3424 3425]
0.000164075408456


In [7]:
### maxvol preprocessing
A_init = np.dot(perm_matrix(p),np.dot(b,perm_matrix(q)))
ids_init = A_init[:A_init.shape[1]]
temp_init = np.dot(A_init,np.linalg.inv(ids_init))

In [8]:
%time r,t,y = block_maxvol(A_init,nder, tol=0.0,max_iters=150,swm_upd=True)

CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 10.7 ms


In [9]:
%time C, B, P = block_maxvol(A_init, nder,tol=0.0,max_iters = 100,swm_upd=False)

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 11 ms


In [10]:
### compare det of initial dominant submatrix with another one after blocked maxvol iterations
print np.linalg.det(ids_init)
#print np.linalg.det(ids)
print np.linalg.det(r[:b.shape[1]])

-7.71141551229
-7.71141551229


In [ ]:
mat_test = np.array([ 
    [50,  0 , 0,0,0, 1],
    [0, 0, 30,  0 ,0, 3],
    [0,0, 0, 20,  0 , 11],
    [0, 40,  0 ,0,0, 2],
    [0,0, 0,0,  10,  12],
    [0,0,0, 0,0, 50]
], dtype=float)
print (mat_test)
print ()

In [ ]:
p,l,u,q, info = pluq_ids(mat_test, debug=True)

In [ ]:
mat_test_3 = np.array([ 
    [20,0, 0, 2, 0 , 11],
    [0, 10,  0 ,0,0.043, 2],    
    [0,  0 , 30,0,15, 1],
    [0, 0, 0,  10 ,0, 0],
    [0,0, 20,0,  90,  12],
    [0,0,0, 0,0, 1900]
], dtype=float)
print (mat_test_3.astype(int))
print ()

In [ ]:
p,l,u,q, info = pluq_ids(mat_test_3, debug=True)

# RGV 2
### Test for linear dependency. 1st thought about pullbacks

In [31]:
mat_test = np.array([
    [ 1000, 0, 0, 0,0,0 ],
    [0,3,0,0,0,1],
    [0, 0 , 1000, 0, 0, 0] , 
    [0, 0, 0, 2, 0, 0],
    [0, 0, 0, 0, 1000, 0] , 
    [0,3,0,2,0,1],
    [1,2,3,4,0,0],
    [4,3,2,1,0,1]
], dtype=float)

## Check, that matrix is good

In [ ]:
np.linalg.matrix_rank(mat_test)

In [ ]:
np.linalg.det(mat_test[[0,1,4,5,6,7]])

In [32]:
p,l,u,q, info = pluq_ids(mat_test, debug=True)
print info

[[ 1000.     0.     0.     0.     0.     0.]
 [    0.     3.     0.     0.     0.     1.]
 [    0.     0.  1000.     0.     0.     0.]
 [    0.     0.     0.     2.     0.     0.]
 [    0.     0.     0.     0.  1000.     0.]
 [    0.     3.     0.     2.     0.     1.]
 [    1.     2.     3.     4.     0.     0.]
 [    4.     3.     2.     1.     0.     1.]]
[[ 1000.     0.]
 [    0.     3.]]
[0 1]
correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 0, 1)
('column coordinates:', array([0, 1], dtype=int32))
('maxvol 2x2 submatrix', array([[ 1000.,     0.],
       [    0.,     3.]]))
('with det = ', 2999.9999999999977)
pivoting and permutations start...
just before elim
[[ 1000.     0.     0.     0.     0.     0.]
 [    0.     3.     0.     0.     0.     1.]
 [    0.     0.  1000.     0.     0.     0.]
 [    0.     0.     0.     2.     0.     0.]
 [    0.     0.     0.     0.  1000.     0.]
 [    0.     3.     0.     2.     0.     1.]
 [    1.     2.     3.     4.     0.     

# Another Error 
### That was about columns swap. 
If piv[0] == j, piv[1] != j+1

In [5]:
mat_test_5 = np.array([[18,  7, 10, 14, 19, 12],
       [16, 15,  5,  6, 17, 15],
       [15,  0,  7,  7, 12, 11],
       [12,  4,  6, 13,  6,  5],
       [19,  8, 10, 16,  3, 16],
       [10,  3,  5,  1,  1,  4],
       [ 0,  7,  7,  6, 17,  9],
       [14,  7,  9, 16, 17,  5],
       [18, 18, 19,  3,  9, 12],
       [12,  7, 16,  1,  0,  2]], dtype=float)

In [14]:
p,l,u,q, info = pluq_ids_pull(mat_test_5, debug=True)

('Jumped on the layer', 0)
black_list now contains entries = 1
diag case
[[  0.  14.]
 [ 17.  17.]]
('on the', 0, 'slice; curr_layer = ', 0)
('best row block is', 6, 7)
('column coordinates:', 0, 4)
('maxvol 2x2 submatrix', array([[  0.,  14.],
       [ 17.,  17.]]))
('with det = ', 237.99999999999986)
pivoting and permutations start...
after 1st pivot
[[ 17.   7.   7.   6.   0.   9.]
 [ 17.  15.   5.   6.  16.  15.]
 [ 12.   0.   7.   7.  15.  11.]
 [  6.   4.   6.  13.  12.   5.]
 [  3.   8.  10.  16.  19.  16.]
 [  1.   3.   5.   1.  10.   4.]
 [ 19.   7.  10.  14.  18.  12.]
 [ 17.   7.   9.  16.  14.   5.]
 [  9.  18.  19.   3.  18.  12.]
 [  0.   7.  16.   1.  12.   2.]]
after 2nd pivot
[[ 17.   0.   7.   6.   7.   9.]
 [ 17.  14.   9.  16.   7.   5.]
 [ 12.  15.   7.   7.   0.  11.]
 [  6.  12.   6.  13.   4.   5.]
 [  3.  19.  10.  16.   8.  16.]
 [  1.  10.   5.   1.   3.   4.]
 [ 19.  18.  10.  14.   7.  12.]
 [ 17.  16.   5.   6.  15.  15.]
 [  9.  18.  19.   3.  18.  12.]
 

In [15]:
p2,l2,u2,q2,inf = pluq_ids(mat_test_5, debug = True)

[[  0.  17.   7.   6.   7.   9.]
 [ 14.  17.   9.  16.   7.   5.]
 [ 15.  12.   7.   7.   0.  11.]
 [ 12.   6.   6.  13.   4.   5.]
 [ 19.   3.  10.  16.   8.  16.]
 [ 10.   1.   5.   1.   3.   4.]
 [ 18.  19.  10.  14.   7.  12.]
 [ 16.  17.   5.   6.  15.  15.]
 [ 18.   9.  19.   3.  18.  12.]
 [ 12.   0.  16.   1.   7.   2.]]
[[  0.  17.]
 [ 14.  17.]]
[1 0]
correct 2x2 matrix
('on the', 0, 'slice')
('best row block is', 6, 7)
('column coordinates:', array([0, 4], dtype=int32))
('maxvol 2x2 submatrix', array([[ 14.,  17.],
       [  0.,  17.]]))
('with det = ', 237.99999999999986)
pivoting and permutations start...
just before elim
[[ 14.  17.   9.  16.   7.   5.]
 [  0.  17.   7.   6.   7.   9.]
 [ 15.  12.   7.   7.   0.  11.]
 [ 12.   6.   6.  13.   4.   5.]
 [ 19.   3.  10.  16.   8.  16.]
 [ 10.   1.   5.   1.   3.   4.]
 [ 18.  19.  10.  14.   7.  12.]
 [ 16.  17.   5.   6.  15.  15.]
 [ 18.   9.  19.   3.  18.  12.]
 [ 12.   0.  16.   1.   7.   2.]]
Elimination starts
after 0

In [16]:
print(la.det(np.dot(perm_matrix(p),mat_test_5)[:6]))
print(la.det(np.dot(perm_matrix(p2),mat_test_5)[:6]))
print p,p2

2395100.0
-2395100.0
[6 7 4 5 8 9 0 1 2 3] [7 6 4 5 8 9 0 1 2 3]


## Yet another bug
About feature of maxvol's performance. It returns error if we have a long pair consisted from more then 2 elements each with rank < 2

In [17]:
mat_test_6 = np.array([[ 3, 13,  4, 11,  6, 11],
       [14, 19, 16,  1,  4,  7],
       [17, 14, 18, 18,  9,  6],
       [ 7,  8,  8,  2, 16, 16],
       [10, 19,  3, 10, 18,  5],
       [ 1, 17,  9, 19, 15, 13],
       [13,  1,  4,  1, 17,  9],
       [10, 13,  8, 11,  3,  7],
       [ 4,  8,  6,  0,  1,  8],
       [ 8, 10, 10,  9,  1, 19]], dtype=float)

In [18]:
p,l,u,q, info = pluq_ids(mat_test_6, debug=True)
p1,l1,u1,q1,inf = pluq_ids_pull(mat_test_6)

[[ 18.   9.  18.  17.  14.   6.]
 [  2.  16.   8.   7.   8.  16.]
 [ 11.   6.   4.   3.  13.  11.]
 [  1.   4.  16.  14.  19.   7.]
 [ 10.  18.   3.  10.  19.   5.]
 [ 19.  15.   9.   1.  17.  13.]
 [  1.  17.   4.  13.   1.   9.]
 [ 11.   3.   8.  10.  13.   7.]
 [  0.   1.   6.   4.   8.   8.]
 [  9.   1.  10.   8.  10.  19.]]
[[ 18.   9.]
 [  2.  16.]]
[0 1]
('on the', 0, 'slice')
('best row block is', 2, 3)
('column coordinates:', array([3, 4], dtype=int32))
('maxvol 2x2 submatrix', array([[ 18.,   9.],
       [  2.,  16.]]))
('with det = ', 275.99999999999989)
pivoting and permutations start...
just before elim
[[ 18.   9.  18.  17.  14.   6.]
 [  2.  16.   8.   7.   8.  16.]
 [ 11.   6.   4.   3.  13.  11.]
 [  1.   4.  16.  14.  19.   7.]
 [ 10.  18.   3.  10.  19.   5.]
 [ 19.  15.   9.   1.  17.  13.]
 [  1.  17.   4.  13.   1.   9.]
 [ 11.   3.   8.  10.  13.   7.]
 [  0.   1.   6.   4.   8.   8.]
 [  9.   1.  10.   8.  10.  19.]]
Elimination starts
after 0 elimination
[[ 18.

In [19]:
print(la.det(np.dot(perm_matrix(p),mat_test_6)[:6]))
print(la.det(np.dot(perm_matrix(p1),mat_test_6)[:6]))
print p,p1

9611254.0
-9611254.0
[2 3 4 5 1 0 6 7 8 9] [2 3 4 5 0 1 6 7 8 9]


## Yet another bug ver. 2.0: Division by Zero
### This and below: again about columns swap. Solved

In [3]:
mat_test_7 = np.array([[ -5,  14, -17, -18,   4,  19],
       [ -1,   0,  16,  -9,   3, -14],
       [ 12,   0, -16,  -4,  -6,  18],
       [ -9,  -4, -17,  12,  14,   0],
       [  0,   2,   0,   0, -15,   0],
       [ 13,  -5,  -8,  -7,  13, -16],
       [ -5,   0,   3,   7,  -7,   0],
       [ 19,  -1, -10,  18,  17,  -9],
       [-15,   6,  -5, -12,   2,   4],
       [-15, -19, -13, -14,   0,  -1]], dtype=float)

In [4]:
p,l,u,q, info = pluq_ids(mat_test_7, debug=True)
p1,l1,u1,q1,inf = pluq_ids_pull(mat_test_7)

441.0
[0 1]
('on the', 0, 'slice')
('best row block is', 0, 1)
('column coordinates:', array([2, 3], dtype=int32))
('maxvol 2x2 submatrix', array([[-17., -18.],
       [ 16.,  -9.]]))
('with det = ', 422.99999999999983)
pivoting and permutations start...
just before elim
Elimination starts
after 0 elimination
after 1 elimination
370.816326531
[0 1]
correct 2x2 matrix
('on the', 2, 'slice')
('best row block is', 4, 5)
('column coordinates:', array([4, 5], dtype=int32))
('maxvol 2x2 submatrix', array([[-15.        ,   0.        ],
       [ 11.50113379, -24.72108844]]))
('with det = ', 370.81632653061223)
pivoting and permutations start...
just before elim
Elimination starts
after 0 elimination
after 1 elimination
-318.498514034
[0 1]
('on the', 4, 'slice')
('best row block is', 8, 9)
('column coordinates:', array([4, 5], dtype=int32))
('maxvol 2x2 submatrix', array([[-20.58741515, -24.38115331],
       [-13.6301596 ,  -0.67130802]]))
('with det = ', 318.49851403412242)
pivoting and permu

In [5]:
print(la.det(np.dot(perm_matrix(p),mat_test_7)[:6]))
print(la.det(np.dot(perm_matrix(p1),mat_test_7)[:6]))
print p,p1
print q,q1

-52084062.0
52084062.0
[0 1 4 5 9 8 6 7 2 3] [0 1 4 5 8 9 6 7 2 3]
[2 3 4 5 0 1] [3 5 4 2 0 1]


## Yet another bug ver. 3.0: Division by Zero

In [31]:
mat_test_8 = np.array([[ -5,  -9, -13,  -2,  19,  11],
       [ 14, -17,  -8,   2, -12,   0],
       [ 17, -19,  -3,   6, -19,   1],
       [ 19,   0, -16, -19,  19,  18],
       [ -5,   2,   1,   4, -17,   1],
       [  4,   2,   7,   9, -10, -15],
       [ 19,  -9,   0, -12,   0,  -6],
       [-10,  15,  12,   2,   2,  13],
       [ 16,   0,   3, -16,  16, -15],
       [-15, -14,   9,   8,   0,  17]], dtype=float)

In [7]:
p,l,u,q, info = pluq_ids(mat_test_8, debug=True)
p1,l1,u1,q1,inf = pluq_ids_pull(mat_test_8)

-684.0
[3 2]
('on the', 0, 'slice')
('best row block is', 2, 3)
('column coordinates:', array([0, 4], dtype=int32))
('maxvol 2x2 submatrix', array([[ 19.,  19.],
       [ 17., -19.]]))
('with det = ', 683.99999999999977)
pivoting and permutations start...
just before elim
Elimination starts
after 0 elimination
after 1 elimination
660.960526316
[3 2]
correct 2x2 matrix
('on the', 2, 'slice')
('best row block is', 8, 9)
('column coordinates:', array([4, 5], dtype=int32))
('maxvol 2x2 submatrix', array([[-21.91666667,  24.91666667],
       [  0.        , -30.15789474]]))
('with det = ', 660.96052631578959)
pivoting and permutations start...
just before elim
Elimination starts
after 0 elimination
after 1 elimination
-115.188791343
[3 2]
('on the', 4, 'slice')
('best row block is', 6, 7)
('column coordinates:', array([4, 5], dtype=int32))
('maxvol 2x2 submatrix', array([[ 22.68723084,   0.68821293],
       [  1.96171176,  -5.01774398]]))
('with det = ', 115.18879134345063)
pivoting and perm

In [8]:
print(la.det(np.dot(perm_matrix(p),mat_test_8)[:6]))
print(la.det(np.dot(perm_matrix(p1),mat_test_8)[:6]))
print p,p1

-52076507.0
52076507.0
[3 2 9 8 7 6 4 5 0 1] [2 3 8 9 6 7 4 5 0 1]


In [9]:
hap = p_preproc(p,2)
print hap

[2 3 8 9 6 7 4 5 0 1]


In [15]:
print(la.det(np.dot(perm_matrix(hap),mat_test_8)[:6]))

52076507.0


In [32]:
indx_n, indx_o = change_intersept(np.arange(4),(np.arange(4))[[1,3,0,2]])
#print indx_n,indx_o
#aab = np.concatenate((indx_n,indx_o))
#bba = np.concatenate((indx_o,indx_n))
#print aab,bba

mat_test_8[indx_n,:] = mat_test_8[indx_o,:]


In [33]:
mat_test_8

array([[ 14., -17.,  -8.,   2., -12.,   0.],
       [ 19.,   0., -16., -19.,  19.,  18.],
       [ -5.,  -9., -13.,  -2.,  19.,  11.],
       [ 17., -19.,  -3.,   6., -19.,   1.],
       [ -5.,   2.,   1.,   4., -17.,   1.],
       [  4.,   2.,   7.,   9., -10., -15.],
       [ 19.,  -9.,   0., -12.,   0.,  -6.],
       [-10.,  15.,  12.,   2.,   2.,  13.],
       [ 16.,   0.,   3., -16.,  16., -15.],
       [-15., -14.,   9.,   8.,   0.,  17.]])